# Sparseloop Tutorial


First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ./prelude.py --style=tree --animation=movie

# Dot product

The following sequence of cells illustrate a dot product computation of:

$$ Z_m = A_k \times B_k $$

## Configure two rank-1 tensors


In [ ]:
#
# Set default shape
#
K = 10
M = 1

#
# Create controls to configure the tensors
#
tm = TensorMaker("sparseloop-dot-product", autoload=True)

tm.addTensor("A", rank_ids=["K"], shape=[K], density=0.4, color="blue")
tm.addTensor("B", rank_ids=["K"], shape=[K], density=1.0, color="green")

tm.displayControls()


## Create and display the tensors

In [ ]:
A_K = tm.makeTensor("A")
B_K = tm.makeTensor("B")

displayTensor(A_K)
displayTensor(B_K)


# Dot product - A, B uncompressed 

In [ ]:
#
# Create the input/output tensors
#
K = getShape(tm, "K")
M = 1

A_K = tm.makeTensor("A")
B_K = tm.makeTensor("B")
Z_M = Tensor(name="Z", rank_ids=["M"], shape=[M])

uncompressTensor(A_K)
uncompressTensor(Z_M)

#
# Display the input tensors
#
print(f"K: {K}")
print(f"M: {M}")

displayTensor(A_K)
displayTensor(B_K)
displayTensor(Z_M)

#
# Get the root fibers of each tensor
#
a_k = A_K.getRoot()
b_k = B_K.getRoot()
z_m = Z_M.getRoot()
#
# Get a reference to the output scalar
#

#
# Animation bookkeeping
#
canvas = createCanvas(A_K, B_K, Z_M)

#
# Traverse all coordinates of shape `K`
#
for m in range(M):

    for k in range(K):
        #
        # The the values at coordinate `k`
        #
        a_val = a_k.getPayload(k)
        b_val = b_k.getPayload(k)
    
        #
        # Do the reduction
        #
        z_m[m] += a_val * b_val
    
        #
        # Animation bookkeeping
        #
        canvas.addActivity([(k,)], [(k,)], [(m,)],
                           spacetime=(0, k))
    
    

displayTensor(Z_M)
displayCanvas(canvas)

# Dot Product - Gating Unneeded Reads

In [ ]:
#
# Create the input/output tensors
#
K = getShape(tm, "K")
M = 1

A_K = tm.makeTensor("A")
B_K = tm.makeTensor("B")
Z_M = Tensor(name="Z", rank_ids=["M"], shape=[M])

uncompressTensor(A_K)
uncompressTensor(Z_M)

#
# Display the input tensors
#
print(f"K: {K}")
print(f"M: {M}")

displayTensor(A_K)
displayTensor(B_K)
displayTensor(Z_M)

#
# Get the root fibers of each tensor
#
a_k = A_K.getRoot()
b_k = B_K.getRoot()
z_m = Z_M.getRoot()

#
# Animation bookkeeping
#
canvas = createCanvas(A_K, B_K, Z_M)

#
# Traverse the single element of the output shape
#
for m in range(M):
    #
    # Traverse all coordinates of shape `K`
    #
    for k in range(K):
        #
        # Get the value of `A` at coordinate `k`
        #
        a_val = a_k.getPayload(k)
    
        #
        # Gate the access to the value of `B`
        # and computation
        #
        if a_val != 0:
            b_val = b_k.getPayload(k)

            z_m[m] += a_val * b_val

        #
        # Animation bookkeeping
        #
        if a_val == 0:
            B_activity = []
            Z_activity = []
        else:
            B_activity = [(k,)]
            Z_activity = [(m,)]
        
        canvas.addActivity([(k,)], B_activity, Z_activity,
                           spacetime=(0, k))
    

displayTensor(Z_M)
displayCanvas(canvas)

## Dot Product - Compressed A

In [ ]:
#
# Create the input/output tensors
#
K = getShape(tm, "K")
M = 1

A_K = tm.makeTensor("A")
B_K = tm.makeTensor("B")
Z_M = Tensor(name="Z", rank_ids=["M"], shape=[M])

uncompressTensor(Z_M)

#
# Display the input tensors
#
print(f"K: {K}")
print(f"M: {M}")

displayTensor(A_K)
displayTensor(B_K)
displayTensor(Z_M)

#
# Get the root fibers of each tensor
#
a_k = A_K.getRoot()
b_k = B_K.getRoot()
z_m = Z_M.getRoot()


#
# Animation bookkeeping
#
canvas = createCanvas(A_K, B_K, Z_M)


for m in range(M):
    #
    # Get a reference to the output scalar
    #
    z_ref = Z_M.getPayloadRef(0)

    #
    # Traverse compressed tensor `A`
    #
    for k, a_val in a_k:
        #
        # Get value of `B` at non-zero coordinates of `A`
        #
        b_val = b_k.getPayload(k)

        z_ref += a_val * b_val

        #
        # Animation bookkeeping
        #
        canvas.addActivity([(k,)], [(k,)], [(m,)],
                           spacetime=(0, k))


displayTensor(Z_M)
displayCanvas(canvas)

## Testing area

For running alternative algorithms